In [ ]:
#In the name of God

#Libraries


student_id = 401210923
student_name = "Navid Farahani"

print("your student id:", student_id)
print("your name:", student_name)

import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import numpy as np
import cv2
from torch.utils.data import Dataset
import json
import time

from torchvision.datasets import CocoDetection
from torch.utils.data.sampler import SubsetRandomSampler


print("For better understanding CNNs, I watched this video from Youtube: https://www.youtube.com/watch?v=d9QHNkD_Pos")
print("also, I read the below article for better understanding deformable CNNs:\n [1] J. Dai et al., Deformable convolutional networks, in Proceedings of the IEEE international conference on computer vision, 2017, pp. 764-773. ")

your student id: 401210923
your name: Navid Farahani
For better understanding CNNs, I watched this video from Youtube: https://www.youtube.com/watch?v=d9QHNkD_Pos
also, I read the below article for better understanding deformable CNNs:
 [1] J. Dai et al., Deformable convolutional networks, in Proceedings of the IEEE international conference on computer vision, 2017, pp. 764-773. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Using COCO Dataset
# We first need to define a class for concatenating the labels and the images.
# I considered val2017 images that are 5000 and have (80 Correct!!!) labels
# because the other labels where empty,
# I save the empty labels as bad_numbers, for example index=250 was empty
# then I used np.unique(labels) and could understand that we have 80 correct labels
# so, in the Neural Network, in the last layer we should have 80 neurons because we have
# 80 classes.
class CustomDataset(Dataset):
    def __init__(self,img_dir,label_dir):

        self.img_dir = img_dir
        self.label_dir=label_dir
        self.img_files = os.listdir(self.img_dir)
        self.num_img = int(len(self.img_files))
        self.coco_dataset = CocoDetection(root=self.img_dir, annFile=self.label_dir)

    def __getitem__(self,index1):
        bad_numbers=[250,370,384,431,516,566,838,862,1015,1066,1501,1524,
         1737,1749,1961,1988,2098,2253,2265,2315,2321,2336,2397,2663,2687,
         2713,2769,2856,2970,3216,3242,3314,3466,3492,3851,3956,3986,4127,4133,
         4169,4344,4439,4557,4623,4659,4731,4779,4809]
        if index1 not in bad_numbers:
            #print(index1)
            # in the COCO dataset, [1][0]['category_id'] represents the label
            label=self.coco_dataset[int(index1)][1][0]['category_id']

            img_file = os.path.join(self.img_dir, self.img_files[index1])
            imggg=cv2.imread(img_file)
            the_imggg=cv2.resize(imggg,(32,32))
        else:
            # For bad numbers, we can consider a non bad number ( for example 0)
            # for the dataset.
            label=self.coco_dataset[int(0)][1][0]['category_id']

            img_file = os.path.join(self.img_dir, self.img_files[0])
            imggg=cv2.imread(img_file)
            the_imggg=cv2.resize(imggg,(32,32))


        return (the_imggg,label)

    def __len__(self):
        return 5000

# Note that the number of the images is 5000, so for __len__ we considered 5000.


# Using COCO dataset from my Google Drive

dataset_train = CustomDataset('drive/MyDrive/val2017','drive/MyDrive/annotations/instances_val2017.json')


num_images=len(dataset_train)
num_train=int(num_images*0.7)
num_validation=int(num_images*0.15)
num_test=int(num_images*0.15)
# Create a DataLoader for the custom dataset
train_dataset,validation_dataset,test_dataset=random_split(dataset_train,[num_train,num_validation,num_test])


batch_size=64
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def to_device(data, device):
    # for every batch, we pass the data to the current device.
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)



class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl=dl
        self.device=device

    def __iter__(self):
        #For every batch, we pass it to the current device.
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

# Selecting the current device
if torch.cuda.is_available():
      device=torch.device('cuda')
else:
      device=torch.device('cpu')

# We move train, validation and test loaders to current device for every batch by using this code
train_dl = DeviceDataLoader(train_loader, device)
val_dl = DeviceDataLoader(val_loader, device)
test_dl = DeviceDataLoader(test_loader, device)


loading annotations into memory...
Done (t=2.02s)
creating index...
index created!


In [ ]:
############# Traditional CNN

# Define the Traditional CNN model
class TraditionalConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(TraditionalConvolutionalNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)# MNIST has just one channel, so the first conv, consider only one layer.
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)# For the last layer which is a FC layer, we just have 10 neurons, because in MNIST we have 10 classes.

    def forward(self, x):
        x=self.conv1(x)
        x=self.relu(x)
        x=self.pool(x)
        x=self.conv2(x)
        x=self.relu(x)
        x=self.pool(x)
        x=x.view(-1, 64 * 7 * 7)  # For Fully Connected Layer, we need to flaten the data.
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x


متاسفانه بنده در این ایمیل، دیتاست را بر روی گوگل درایو کرده بودم اما جی پی یوی آن تمام شد و شبیه سازی نهایی انجام نشد! اما دیتاست لود شده است؛ ضمناً این تمرین با دیتاست ام نیست شبیه سازی شد.

In [ ]:
model=TraditionalConvolutionalNetwork()


model.to(device)

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

# Training loop
num_epochs = 10
accuracy=np.zeros(num_epochs)
losses=np.zeros(num_epochs)

tic=time.time()
for epoch in range(num_epochs):
    model.train()
    for images,labels in train_dl:
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        acc=0
        all_samples=0
        for images,labels in val_dl:
            images=images.to(device)
            labels=labels.to(device)
            outputs=model(images)
            _,predicted=torch.max(outputs,1)
            all_samples=all_samples+labels.size(0)
            acc=acc+(predicted==labels).sum().item()

        accuracy[epoch] = acc / all_samples
        losses[epoch]=loss
        print("Epoch ",epoch,"\nLoss:",round(losses[epoch],4),"Accuracy:",round(accuracy[epoch],4))

toc=time.time()



with torch.no_grad():
    acc=0
    all_samples=0
    for images,labels in val_dl:
          images=images.to(device)
          labels=labels.to(device)
          outputs=model(images)
          _,predicted=torch.max(outputs,1)
          all_samples=all_samples+labels.size(0)
          acc=acc+(predicted==labels).sum().item()

    accuracy_for_test = acc / all_samples
    print("Accuracy for Test:",round(accuracy_for_test,4))



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-33e12c848be2>", line 18, in <cell line: 16>
    for images,labels in train_dl:
  File "<ipython-input-3-58ae9e9afadd>", line 82, in __iter__
    for b in self.dl:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 674, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataset.py", line 364, in __getitems__
    return [self.dataset[self.indices[idx

TypeError: ignored

In [ ]:
Traditional_required_time=toc-tic
Accuracy_Traditional=accuracy
Losses_Traditional=losses
Accuracy_Traditional_test=accuracy_for_test


print("Required time for simulation for Traditional CNN:"+str(Traditional_required_time)+"s")
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(accuracy)
plt.title("Accuracy Traditional")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")

plt.subplot(2,1,2)
plt.plot(losses)
plt.title("Losses Traditional")
plt.ylabel("Loss")
plt.xlabel("Epochs")

In [ ]:
### Deformable CNN

# As I mentioned in the report of theoric questions, we need to consider offsets as some parameters and multiple these to
# the position of the samples of the grid and displace them.
# These offsets are learnable and during the training, we train it in an end-to-end form.
class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(DeformableConv2d, self).__init__()

        self.kernel_size = kernel_size
        self.padding = padding


        # We can consider 9 offsets for all samples of the kernels(kernels are 3 by 3) for all input channels. If the grid is h * w, we have h*w grid samples.
        # So we can update these parameters if we consider these samples as a conv layer with an arbitrary stride and padding and kernel size.
        # In fact we train the position of the offsets by help of a specific nn.Conv2d and we use for all possible kernels due to striding and paddings and finally
        # we multiple this grids to the Traditional Kernel.

        # Note that we have 2 offests for x and y directs, so the output channels must be 2*kernel_size*kernel_size.

        self.conv_offset = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size, kernel_size=kernel_size, stride=stride, padding=padding)


        # We define conv_weight for the image for extracting features. But in the forward method, we multiple the offsets to this weights
        # And we finally use this weights instead of the module nn.Conv2d for extracting features in a layer.

        self.conv_weight = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)

        # We use this function for passing the scattered grid and find the best position by extracting features by using a nn.Conv2d as
        self.Conv_For_Grid= nn.Conv2d(kernel_size * kernel_size, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)

    def forward(self, x):
        #Calculating offsets
        offsets = self.conv_offset(x)

        # Generate grid for sampling
        # For every batch and channel, we consider grid_y and grid_x to construct the structured grid(they don't have irregular shapes yet)

        grid_y=offsets[:,:self.kernel_size*self.kernel_size, :, :]
        grid_x=offsets[:,self.kernel_size*self.kernel_size:, :, :]
        #Size of offests: (batchs,kernel**2,h,w)

        # Normalize grid to range [-1, 1]
        # According to the referenced article, we can define the R region in the interval of [-1,1], so if we normalize it we can construct this R
        grid_y = (2 * grid_y / (x.size(-2) - 1)) - 1
        grid_x = (2 * grid_x / (x.size(-1) - 1)) - 1

        sampled_features=(grid_x+grid_y)/2

        # After constructing the grid, we pass it from a network and it will be trained, so we can multiple this offsets to the
        # traditional kernel which is called "weights".
        sampled_features=self.Conv_For_Grid(sampled_features)
        weights = self.conv_weight(x)
        output = sampled_features * weights

        return output





In [ ]:
# Define the CNN model
class DeformableConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(DeformableConvolutionalNetwork, self).__init__()

        self.conv1=DeformableConv2d(1, 32, kernel_size=3, padding=1)
        self.relu=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2=DeformableConv2d(32, 64, kernel_size=3, padding=1)
        self.fc1=nn.Linear(64 * 7 * 7, 128)
        self.fc2=nn.Linear(128, 10)

    def forward(self, x):
        x=self.conv1(x)
        x=self.relu(x)
        x=self.pool(x)
        x=self.conv2(x)
        x=self.relu(x)
        x=self.pool(x)
        x=x.view(-1, 64 * 7 * 7)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x

model = DeformableConvolutionalNetwork()

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
accuracy=np.zeros(num_epochs)
losses=np.zeros(num_epochs)

tic=time.time()
for epoch in range(num_epochs):
    model.train()
    for images,labels in train_dl:
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        acc=0
        all_samples=0
        for images,labels in val_dl:
            images=images.to(device)
            labels=labels.to(device)
            outputs=model(images)
            _,predicted=torch.max(outputs,1)
            all_samples=all_samples+labels.size(0)
            acc=acc+(predicted==labels).sum().item()

        accuracy[epoch] = acc / all_samples
        losses[epoch]=loss
        print("Epoch ",epoch,"\nLoss:",round(losses[epoch],4),"Accuracy:",round(accuracy[epoch],4))

toc=time.time()



with torch.no_grad():
    acc=0
    all_samples=0
    for images,labels in val_dl:
          images=images.to(device)
          labels=labels.to(device)
          outputs=model(images)
          _,predicted=torch.max(outputs,1)
          all_samples=all_samples+labels.size(0)
          acc=acc+(predicted==labels).sum().item()

    accuracy_for_test = acc / all_samples
    print("Accuracy for Test:",round(accuracy_for_test,4))


In [ ]:
Deformable_required_time=toc-tic
Accuracy_Deformable=accuracy
Losses_Deformable=losses
Accuracy_Deformable_test=accuracy_for_test

print("Required time for simulation for Deformable CNN:"+str(Deformable_required_time)+"s")
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(accuracy)
plt.title("Accuracy Deformable")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")

plt.subplot(2,1,2)
plt.plot(losses)
plt.title("Losses Deformable ")
plt.ylabel("Loss")
plt.xlabel("Epochs")


In [ ]:
############# Comparing Two Methods
print("Accuracy for Deformable is  "+str(100*float(Accuracy_Deformable[-1]-Accuracy_Traditional[-1]))+"%  more than traditional CNN")
print("Required time for Deformable is  "+str(round(float(Deformable_required_time-Traditional_required_time),4))+"s  more than traditional CNN")
